In [29]:
import pandas as pd
import yfinance as yf
import numpy as np
from get_data import get_req_data
import yahooquery as yq

In [30]:
# Param configuration
start_date = "2015-01-08"
end_date = "2015-01-15"

pd.set_option('display.max_rows', 20)

#Sector Map
sector_map = {
'S&P 500 Materials (Sector)' : 'basic_materials', 
'S&P 500 Industrials (Sector)' : 'industrials',
'S&P 500 Consumer Discretionary (Sector)' : 'consumer_cyclical' ,
'S&P 500 Consumer Staples (Sector)' : 'consumer_defensive',
'S&P 500 Health Care (Sector)' : 'healthcare', 
'S&P 500 Financials (Sector)' : 'financial_services',
'S&P 500 Information Technology (Sector)' : 'technology',
'S&P 500 Telecommunication Services (Sector)' : 'communication_services',
'S&P 500 Utilities (Sector)' : 'utilities', 
'S&P 500 Real Estate (Sector)' : 'realestate',
'S&P 500 Real Enegry (Sector)' : 'energy'
}


In [35]:
# Generate data frame to hold all required information
sectors_ticker = "SPY"

# Load in info
sp500_tickers_data = get_req_data(start_date, end_date)
weights = yq.Ticker(sectors_ticker).fund_sector_weightings.to_dict()

# add the sectors + dates + adj close to the dataframe
records = []
for sector, data in sp500_tickers_data.items():
    for x in range(0, len(data['sector_data'])):

        record = {"sector" : sector, "date" : data['sector_data'].index[x], "adj_close" : data["sector_data"][x], "sector_weight" : weights[sectors_ticker][sector_map[sector]] }
        records.append(record)

df = pd.DataFrame(records)

df["pct_change"] = df.groupby("sector")["adj_close"].pct_change()

df["contribution"] = df["pct_change"] * df["sector_weight"]

print(weights)
display(df)


{'SPY': {'realestate': 0.028900001, 'consumer_cyclical': 0.11270001, 'basic_materials': 0.0222, 'consumer_defensive': 0.070199996, 'technology': 0.2525, 'communication_services': 0.0841, 'financial_services': 0.128, 'utilities': 0.024500001, 'industrials': 0.083500005, 'energy': 0.0437, 'healthcare': 0.1439}}


,sector,date,adj_close,sector_weight,pct_change,contribution
0,S&P 500 Materials (Sector),2015-01-07,298.519989,0.0222,NaN,NaN
1,S&P 500 Materials (Sector),2015-01-08,305.600006,0.0222,0.023717,0.000527
2,S&P 500 Materials (Sector),2015-01-09,304.070007,0.0222,-0.005007,-0.000111
3,S&P 500 Materials (Sector),2015-01-12,303.130005,0.0222,-0.003091,-0.000069
4,S&P 500 Materials (Sector),2015-01-13,299.609985,0.0222,-0.011612,-0.000258
...,...,...,...,...,...,...
55,S&P 500 Real Estate (Sector),2015-01-08,196.839996,0.0289,0.005055,0.000146
56,S&P 500 Real Estate (Sector),2015-01-09,197.130005,0.0289,0.001473,0.000043
57,S&P 500 Real Estate (Sector),2015-01-12,198.130005,0.0289,0.005073,0.000147
58,S&P 500 Real Estate (Sector),2015-01-13,197.559998,0.0289,-0.002877,-0.000083


In [32]:
contributions = df.groupby("sector").agg({"contribution": "sum"})
contributions["contribution_as_pct"] = (contributions["contribution"] / df["contribution"].sum())*100
contributions

,contribution,contribution_as_pct
sector,,
S&P 500 Consumer Discretionary (Sector),-0.001575,50.074753
S&P 500 Consumer Staples (Sector),0.000061,-1.944188
S&P 500 Financials (Sector),-0.003159,100.415128
S&P 500 Health Care (Sector),0.000374,-11.873611
S&P 500 Industrials (Sector),-0.000437,13.885528
S&P 500 Information Technology (Sector),0.000129,-4.105739
S&P 500 Materials (Sector),-0.000175,5.557621
S&P 500 Real Estate (Sector),0.000419,-13.315923
S&P 500 Telecommunication Services (Sector),0.000960,-30.514046


In [18]:
# weights

{'SPY': {'realestate': 0.028900001,
  'consumer_cyclical': 0.11270001,
  'basic_materials': 0.0222,
  'consumer_defensive': 0.070199996,
  'technology': 0.2525,
  'communication_services': 0.0841,
  'financial_services': 0.128,
  'utilities': 0.024500001,
  'industrials': 0.083500005,
  'energy': 0.0437,
  'healthcare': 0.1439}}

In [34]:
# yq.Ticker(sectors_ticker).fund_sector_weightings.SPY.sum()

0.994200013